# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2

ModuleNotFoundError: No module named 'pymorphy2'

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [ ]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

## Лабораторная работа 9

In [86]:
import pandas as pd
import numpy as np
import nltk

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [87]:
data = pd.read_csv('preprocessed_descriptions.csv')

data.head()

,name,preprocessed_descriptions
0,george s at the cove black bean soup,an original recipe created by chef scott meska...
1,healthy for them yogurt popsicles,my children and their friends ask for my homem...
2,i can t believe it s spinach,these were so go it surprised even me
3,italian gut busters,my sisterinlaw made these for us at a family g...
4,love is in the air beef fondue sauces,i think a fondue is a very romantic casual din...


In [88]:
data = data.preprocessed_descriptions

In [90]:
n = list()

for text in data:
    try:
        n += [x for x in nltk.word_tokenize(text) if x.isalpha()]
    except TypeError as e:
        pass
    
arr = np.array(n)

arr = np.unique(arr)

arr

array(['a', 'aaaahhhhhamazing', 'aaaahing', ..., 'zwtii', 'zwtlaos',
       'zzar'], dtype='<U100')

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [91]:
from nltk.metrics.distance import edit_distance

In [92]:
sample = np.random.choice(arr, size=(5, 2))

In [93]:
for s1, s2 in sample:
    print(f"Edit distance between '{s1}' and '{s2}' is {edit_distance(s1, s2, substitution_cost=2)}")

Edit distance between 'coconut' and 'fireplace' is 14
Edit distance between 'originated' and 'grenada' is 9
Edit distance between 'preservative' and 'slim' is 12
Edit distance between 'deficient' and 'redorange' is 14
Edit distance between 'vibrant' and 'marino' is 9


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [94]:
def k_nearest(word, k, words):
    sorted_words = sorted(words, key=lambda x: edit_distance(word, x))
    return sorted_words[1:k+1]

In [95]:
random_word = np.random.choice(arr)
random_word

'accompany'

In [96]:
k_nearest(random_word, 10, arr)

['company', 'accompanied', 'accompanies', 'accompaning', 'accompanying', 'companys', 'accomadate', 'accomodate', 'accompanyment', 'accomplish']

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [97]:
from nltk.stem import SnowballStemmer, WordNetLemmatizer

In [98]:
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()

stems = list()
lems = list()

for word in arr:
    stems.append(stemmer.stem(word))
    lems.append(lemmatizer.lemmatize(word))
    


In [99]:
ser = pd.Series(stems,index=arr)
df = pd.DataFrame(data=ser, columns=['stemmed_word'])
df['normalized_word'] = lems
df

,stemmed_word,normalized_word
a,a,a
aaaahhhhhamazing,aaaahhhhhamaz,aaaahhhhhamazing
aaaahing,aaaah,aaaahing
aaahs,aaah,aaahs
aaar,aaar,aaar
...,...,...
zwarte,zwart,zwarte
zwt,zwt,zwt
zwtii,zwtii,zwtii
zwtlaos,zwtlao,zwtlaos


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [100]:
n = list()

for text in data:
    try:
        n += [x for x in nltk.word_tokenize(text) if x.isalpha()]
    except TypeError as e:
        pass
    
arr = np.array(n)

In [101]:
from nltk.probability import FreqDist

fdist = FreqDist(arr)

In [102]:
fdist.most_common(10)

[('the', 40072), ('a', 34951), ('and', 30245), ('this', 26859), ('i', 24836), ('to', 23471), ('is', 20285), ('it', 19756), ('of', 18364), ('for', 15939)]

In [103]:
from nltk.corpus import stopwords

In [104]:
eng_stop = set(stopwords.words('english'))

In [105]:
cleared_arr = [x for x in arr if x not in eng_stop]

sdist = FreqDist(cleared_arr)

In [106]:
sdist.most_common(10)

[('recipe', 14871), ('make', 6326), ('time', 5137), ('use', 4620), ('great', 4430), ('like', 4167), ('easy', 4152), ('one', 3872), ('made', 3810), ('good', 3791)]

In [107]:
len(cleared_arr) / len(arr)

0.5386528718649717

### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [108]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [109]:
df_sample = pd.read_csv('preprocessed_descriptions.csv').sample(5)
df_sample

,name,preprocessed_descriptions
27933,totally tropical genuine rum punch,just inflate a plastic palm tree blow up a pla...
3864,broccolini ricotta pasta,i just saw this pasta being made on rachel ray...
11389,fresh mint ice cream,there are no words that can describe fresh hom...
23768,seafood spaghetti,heres a delicious spaghetti recipe this is a w...
20985,platanos maduros costa rican fried ripe plant...,i learned to make these from my friend adrin m...


In [110]:
sample = df_sample.preprocessed_descriptions
sample

27933    just inflate a plastic palm tree blow up a pla...
3864     i just saw this pasta being made on rachel ray...
11389    there are no words that can describe fresh hom...
23768    heres a delicious spaghetti recipe this is a w...
20985    i learned to make these from my friend adrin m...
Name: preprocessed_descriptions, dtype: object

In [111]:
tv = TfidfVectorizer()

corpus_tv = tv.fit_transform(sample)

In [112]:
tv.get_feature_names_out()

array(['30', 'adrin', 'almost', 'although', 'and', 'approximate', 'are',
       'as', 'bananas', 'be', 'beans', 'being', 'black', 'blow', 'bread',
       'brush', 'but', 'by', 'can', 'close', 'college', 'cooking',
       'costa', 'cream', 'crusty', 'dancing', 'decided', 'delicious',
       'describe', 'dessertlikeyou', 'different', 'dish', 'do', 'easy',
       'eaten', 'enough', 'eyes', 'fast', 'favorite', 'find', 'fish',
       'fixes', 'for', 'frequently', 'fresh', 'freshest', 'friend',
       'from', 'gallo', 'garlic', 'gave', 'go', 'goand', 'got', 'great',
       'hammock', 'hang', 'have', 'he', 'heres', 'hes', 'him', 'his',
       'homemade', 'hope', 'household', 'ice', 'imagine', 'in', 'include',
       'inflate', 'instructions', 'is', 'it', 'its', 'just', 'keeping',
       'kitchen', 'learned', 'like', 'limbo', 'local', 'look', 'love',
       'low', 'made', 'make', 'marchena', 'may', 'me', 'meal', 'meals',
       'meat', 'mint', 'minute', 'monger', 'monkey', 'more', 'most', 'my'

In [113]:
vectors = corpus_tv.toarray()
vectors

array([[0.        , 0.        , 0.        , 0.        , 0.08850685,
        0.        , 0.08850685, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.15709903, 0.        ,
        0.15709903, 0.        , 0.        , 0.08850685, 0.15709903,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.15709903, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.15709903, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.15709903, 0.        , 0.        ,
        0.15709903, 0.15709903, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.15709903, 0.        , 0.        ,
        0.15709903, 0.        , 0.        , 0.  

pairwise_similarity = corpus_tv * corpus_tv.T
pairwise_similarity.toarray()

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [114]:
import scipy

In [115]:
correlation = [[0 for _ in range(5)] for _ in range(5)]

In [116]:
for i in range(len(vectors)):
    for j in range(len(vectors)):  
        correlation[i][j] = scipy.spatial.distance.cosine(vectors[i], vectors[j])

In [117]:
new_df = pd.DataFrame(data=correlation, index=df_sample.name, columns=df_sample.name)

In [118]:
new_df

name,totally tropical genuine rum punch,broccolini ricotta pasta,fresh mint ice cream,seafood spaghetti,platanos maduros costa rican fried ripe plantains
name,,,,,
totally tropical genuine rum punch,0.000000,0.884133,0.968594,0.861273,0.961359
broccolini ricotta pasta,0.884133,0.000000,1.000000,0.845766,0.898026
fresh mint ice cream,0.968594,1.000000,0.000000,0.963063,0.947588
seafood spaghetti,0.861273,0.845766,0.963063,0.000000,0.817558
platanos maduros costa rican fried ripe plantains,0.961359,0.898026,0.947588,0.817558,0.000000


3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).

### Наиболее похожими являются те рецепты, у которых косинусное расстояние наименьшое (стремится к нулю).
### На данной выборке минимальное косинусное расстояние у рецептов платанос мадурос) и сифуд спагети